### Tabnet

In [39]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import time
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import torch

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [40]:
# 데이터 불러오기
x_train = pd.read_csv("C:/Users/sumin/Bitamin/2학기/conference/bitamin_ChurnProject-1/data/fill_data/x_train.csv")
x_valid = pd.read_csv("C:/Users/sumin/Bitamin/2학기/conference/bitamin_ChurnProject-1/data/fill_data/x_valid.csv")
y_train = pd.read_csv("C:/Users/sumin/Bitamin/2학기/conference/bitamin_ChurnProject-1/data/fill_data/y_train.csv")
y_valid = pd.read_csv("C:/Users/sumin/Bitamin/2학기/conference/bitamin_ChurnProject-1/data/fill_data/y_valid.csv")

In [41]:
x_train = x_train.values
x_valid = x_valid.values
y_train = y_train.values
y_valid = y_valid.values

In [42]:
y_train = np.array(y_train).ravel()
y_valid = np.array(y_valid).ravel()

modeling

In [56]:
start_time = time.time()
clf = TabNetClassifier(
    n_d=56, 
    n_steps=3, gamma=1.4,
    lambda_sparse=1.1694994589239397e-05, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type='sparsemax',
    n_shared=2,
    scheduler_params=dict(mode="min",
        patience = 3, # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=0)
clf.fit(x_train, y_train,
    eval_set=[(x_train, y_train),(x_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    drop_last = False,
    patience = 29, max_epochs = 80
)
end_time = time.time()
print('걸린 시간 :', end_time - start_time)

## 기본값: 0.81477


Early stopping occurred at epoch 39 with best_epoch = 10 and best_valid_accuracy = 0.82578


c:\Users\sumin\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


걸린 시간 : 869.9879610538483


In [57]:
# prediction and evaluation
# train data
train_pred_y = clf.predict(x_train)
print('Accuracy: {:.2f}'.format(accuracy_score(y_train, train_pred_y)))
print(confusion_matrix(y_train, train_pred_y))
print(classification_report(y_train, train_pred_y))
print(f1_score(y_train, train_pred_y, average='micro'))

# valid data
valid_pred_y = clf.predict(x_valid)
print('Accuracy: {:.2f}'.format(accuracy_score(y_valid, valid_pred_y)))
print(confusion_matrix(y_valid, valid_pred_y))
print(classification_report(y_valid, valid_pred_y))
print(f1_score(y_valid, valid_pred_y, average='micro'))

Accuracy: 0.83
[[ 2412 16731]
 [  336 79678]]
              precision    recall  f1-score   support

        -1.0       0.88      0.13      0.22     19143
         1.0       0.83      1.00      0.90     80014

    accuracy                           0.83     99157
   macro avg       0.85      0.56      0.56     99157
weighted avg       0.84      0.83      0.77     99157

0.8278790201397784
Accuracy: 0.83
[[ 190 1361]
 [  31 6408]]
              precision    recall  f1-score   support

        -1.0       0.86      0.12      0.21      1551
         1.0       0.82      1.00      0.90      6439

    accuracy                           0.83      7990
   macro avg       0.84      0.56      0.56      7990
weighted avg       0.83      0.83      0.77      7990

0.8257822277847309


하이퍼파라미터 튜닝

In [45]:
def Objective(trial : Trial, x_train, y_train, x_valid, y_valid):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
        lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        mask_type=mask_type, n_shared=n_shared,
        scheduler_params=dict(mode="min",
        patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience
        min_lr=1e-5,factor=0.5,),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose=0,
        ) #early stopping
    
    clf = TabNetClassifier(**tabnet_params)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
        patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
        eval_metric=['accuracy'])

    score = f1_score(clf.predict(x_valid), y_valid, average='micro')
    
    return score 


In [46]:
study = optuna.create_study(direction="maximize", sampler = TPESampler(), study_name='TabNet optimization')
study.optimize(lambda trial : Objective(trial, x_train, y_train, x_valid, y_valid), n_trials = 10)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

### Best trial : score 0.8359198998748436, 
### params {'mask_type': 'sparsemax', 'n_da': 56, 'n_steps': 3, 'gamma': 1.4, 'n_shared': 2, 'lambda_sparse': 1.1694994589239397e-05, 'patienceScheduler': 3, 'patience': 29, 'epochs': 80}

[I 2023-01-30 15:36:09,979] A new study created in memory with name: TabNet optimization



Early stopping occurred at epoch 34 with best_epoch = 9 and best_val_1_accuracy = 0.83492


c:\Users\sumin\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-30 15:59:27,052] Trial 0 finished with value: 0.8349186483103879 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 5.39549495923216e-06, 'patienceScheduler': 10, 'patience': 25, 'epochs': 41}. Best is trial 0 with value: 0.8349186483103879.



Early stopping occurred at epoch 24 with best_epoch = 6 and best_val_1_accuracy = 0.83091


c:\Users\sumin\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-30 16:11:32,665] Trial 1 finished with value: 0.8309136420525657 and parameters: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 2.8289851088977512e-05, 'patienceScheduler': 6, 'patience': 18, 'epochs': 73}. Best is trial 0 with value: 0.8349186483103879.


...

Stop training because you reached max_epochs = 4 with best_epoch = 3 and best_val_1_accuracy = 0.81289


c:\Users\sumin\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-30 17:18:33,168] Trial 8 finished with value: 0.8128911138923655 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.4, 'n_shared': 2, 'lambda_sparse': 2.4444596600446757e-05, 'patienceScheduler': 9, 'patience': 26, 'epochs': 4}. Best is trial 4 with value: 0.8359198998748436.



Early stopping occurred at epoch 26 with best_epoch = 3 and best_val_1_accuracy = 0.81527


c:\Users\sumin\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-01-30 17:31:47,669] Trial 9 finished with value: 0.8152690863579475 and parameters: {'mask_type': 'sparsemax', 'n_da': 60, 'n_steps': 2, 'gamma': 1.4, 'n_shared': 3, 'lambda_sparse': 6.719314541450749e-06, 'patienceScheduler': 8, 'patience': 23, 'epochs': 70}. Best is trial 4 with value: 0.8359198998748436.


Best trial : score 0.8359198998748436, 
params {'mask_type': 'sparsemax', 'n_da': 56, 'n_steps': 3, 'gamma': 1.4, 'n_shared': 2, 'lambda_sparse': 1.1694994589239397e-05, 'patienceScheduler': 3, 'patience': 29, 'epochs': 80}
